In [11]:
# Step 1: Import necessary modules
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile, assemble
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import numpy as np

# Step 2: Define the qft function
def qft(n, invert, reverse):
    qc = QuantumCircuit(n)
    for i in reversed(range(n)):
        qc.h(i)
        for j in reversed(range(i)):
            qc.cp(np.pi * (2.0 ** (j - i)), j, i)
    if reverse:
        for i in range(n // 2):
            qc.swap(i, n - i - 1)
    if invert:
        qc = qc.inverse()
    return qc

# Step 3: Define the adder function
def adder(a: QuantumRegister, b: QuantumRegister):
    n = len(a)
    qc = QuantumCircuit(a, b)
    qc.append(qft(n, invert=False, reverse=False).to_gate(), b)
    for j in range(n):
        for k in range(j + 1):
            qc.cp(2 * np.pi / (2 ** (j - k + 1)), a[k], b[j])
    qc.append(qft(n, invert=True, reverse=False).to_gate(), b)
    return qc


In [21]:
a = 7
b = 7
n = 4

qr_a = QuantumRegister(n)
qr_b = QuantumRegister(n)

cr = ClassicalRegister(n, 'cr')

qc = QuantumCircuit(qr_a, qr_b, cr)

for i in range(n):
    if a & (1 << i):
        qc.x(qr_a[i])
        
for i in range(n):
    if b & (1 << i):
        qc.x(qr_b[i])

qc.compose(adder(qr_a, qr_b), inplace=True)

qc.measure(qr_b, cr)

# Execute the circuit
sim = AerSimulator()
t_qc = transpile(qc, sim)
result = sim.run(t_qc, shots=1, memory=True).result()
counts = result.get_memory()[0]

print("Measurement result (binary):", counts)

Measurement result (binary): 1110
